In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, array, explode, struct

In [2]:
spark = SparkSession.builder.appName('Rank_Mean').getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Rank_Mean>

In [3]:
## por temas de replicación, siempre es mejor convertir el archivo a leer en formato csv

monthly_data = pd.read_csv("data-resources/monthly_data.csv")
daily_data = pd.read_csv("data-resources/daily_data.csv")

In [4]:
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ése índice.

benchmark_month = monthly_data.loc[0:monthly_data.shape[0], monthly_data.columns[1]]
benchmark_day = daily_data.loc[0:daily_data.shape[0], daily_data.columns[1]]

In [5]:
## Calculamos el porcentaje de cambio de un día contra otro.
## fórmula: (t+1 / t)-1
## pseudo-código: (precio_hoy / precio_ayer)-1

pct_benchmark_month = benchmark_month.pct_change(1)
pct_benchmark_day = benchmark_day.pct_change(1)

In [6]:
## el vector de "percentage change" se convierte a un arreglo numpy de dimensión (147,)
## NOTA: en los arreglos (objetos) de tipo numpy.array preservan los valores, tanto por fila, como por columna,
##       el órden de los elementos, cómo un 'índice implícito'

pct_benchmark_month_array = np.array(pct_benchmark_month)
pct_benchmark_day_array = np.array(pct_benchmark_day)

In [7]:
## Lo mismo hacemos, pero para el resto de variables equity,
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ese índice.

investment_universe_month = monthly_data.loc[0:monthly_data.shape[0],monthly_data.columns[2:monthly_data.shape[1]]]
investment_universe_day = daily_data.loc[0:daily_data.shape[0],daily_data.columns[2:daily_data.shape[1]]]

In [8]:
## (t+1 / t)-1
## (precio_hoy / precio_ayer)-1

pct_investment_month = investment_universe_month.pct_change(1)
pct_investment_day = investment_universe_day.pct_change(1)

In [9]:
## el vector de percentage change se convierte a un arreglo numpy de dimensión (147,70)

pct_investment_month_array = np.array(pct_investment_month)
pct_investment_day_array = np.array(pct_investment_day)

In [10]:
## creamos arreglos numpy con dimensiones X+1 = 148, rellenas de ceros, para ser imputados con nuevos vectores

up_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
down_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
up_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))
down_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))

In [11]:
## rellenamos las matrices de ceros con valores que aprueben las condiciones, 
## se realiza una comparación dentro de los arreglos de porcentajes de cambio, 
## sí alguno de esos porcentajes es superior a 0, entonces entra a los arreglos 
## de movimientos positivos (incrementos), pero sí alguno es menor que 0, entonces
## el porcentaje se almacena en los arreglos de movimientos negativos (decrementos).

## Básicamente, se separan los porcentajes de cambio en dos matrices: 
## matriz de positivos cuando el porcentaje es > 0 
## matriz de negativos cuando el porcentaje es <= 0

size_benchmark_matrix = pct_benchmark_month_array.shape[0]
for i in range (1, size_benchmark_matrix):
    if pct_benchmark_month_array[i] > 0:
        up_month[i] = pct_benchmark_month_array[i]
        up_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]
    else:
        down_month[i] = pct_benchmark_month_array[i]
        down_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]

In [12]:
## calculamos los vectores 'peor más alto' y 'mejor más alto'

np.seterr(divide='ignore', invalid='ignore')
greater_worse = down_move / down_month
greater_better = (up_move / up_month) * float(-1.0)

In [13]:
## ambos vectores los convertimos a pandas dataframes, y solo nos quedamos con los vectores que tengan valores != np.nan
## una de las ventajas de los pandas dataframes es que mantienen un ídince único por row, esto lo hace poder separarse, y juntarse
## en cuantos sub-conjuntos se requieran y siempre se podrá mantener un órden.

greater_worse_df = pd.DataFrame(data=greater_worse).dropna()
greater_better_df = pd.DataFrame(data=greater_better).dropna()

In [14]:
## calculamos ahora, la mediana acumulada con los pandas dataframes que construimos, 
## con un periodo mínimo (método expanding) de al menos 1 observación dada.

median_down = greater_worse_df.expanding().median()
median_up = greater_better_df.expanding().median()

In [15]:
# se transponen ambos pandas df por la columna periodos, columna que almacena números no consecutivos desde 1 hasta 147

down_transpose = median_down.T
up_transpose = median_up.T

In [16]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_down

ranked_down = down_transpose.rank()
transpose_ranked_down = ranked_down.T

In [17]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_up

ranked_up = up_transpose.rank()
transpose_ranked_up = ranked_up.T

In [18]:
## se añade variable 'label' con la idea de que al juntar ambos dataframes se puedan distinguir los 'worse' de los 'better'
## y se unen ambos dataframes con la etiqueta creada, se usó el método 'insert' por lo que no se deberá correr de nuevo, una
## vez ejecutado ya que fallará por duplicidad de columnas.

worse_better_df = pd.concat([transpose_ranked_up, transpose_ranked_down]).sort_index()

In [19]:
## se crea un índice 'closing_id' para cada registro, éste corre de [1:N] 
## con la idea de etiquetar el id del mes de registro de cierre,
## de la misma forma que lo anterior, NO se deberá ejecutar de nuevo; una vez hecho.

worse_better_df['closing_id'] = range(1, len(worse_better_df) + 1)

In [20]:
## se transponen ambos dataframes, de antes tener una dimensión (68, 70), es decir; 
## 68 registros i.e. 'Rows' (variables)
## 70 columnas fijas (a menos que sea añadido otro asset desde el csv inicial)

## a tener una dimensión 'transpuesta' (invertída sí querés...) de (70, 68), es decir;
## 70 registros i.e. 'Rows' fijos (a menos que sea añadido otro asset desde el csv inicial)
## 68 columnas (variables!!!)

In [21]:
## de pandas dataframes, una vez separados en dos conjuntos ['worse', 'better'],
## creamos por separado dos spark dataframes.

worse_better = spark.createDataFrame(worse_better_df)

In [23]:
## se crea un apuntador "shape long format", una lista con iteraciones, esta lista trabajará con dos variables principales,
## 1-. la variable 'equity_index', será la que contenga los 'id' de los activos
## 2-. la variable 'median_down', será la que contenga la mediana acumulada por cada activo.
## Se mantendrá a lo largo de la transformación 1 columna fija; 'closing_id',
## closing_id: variable que indica el mes de cierre y reporte de precio

def shape_long_format(dataframe, pivot_col):
    
    columns, data_type = zip(*((c, t) for (c, t) in dataframe.dtypes if c not in pivot_col))
    assert len(set(data_type)) == 1, "Columns not the same data type..."
    
    column_explode = explode(array([
        struct(lit(c).alias("asset_id"), col(c).alias("top_rank")) for c in columns
    ])).alias("column_explode")

    return dataframe.select(pivot_col + [column_explode]).select(pivot_col + ["column_explode.asset_id", "column_explode.top_rank"])

In [24]:
## se crea nuevo spark-dataframe donde solo se mostrará por partición ['closing_id'] 
## el top 10 mejores meses donde tuvo menos malos que el resto de los registros; ["top_rank"].

asset_ranking_df = shape_long_format(worse_better, ["closing_id"]).where(col("top_rank") <= 10).orderBy("closing_id", "top_rank")

# A partir de aqui se escribe en formato csv para trabajar con Pandas

In [26]:
## Aqui es donde escribo la tabla en formato csv.

asset_ranking_df.coalesce(1).write.mode('overwrite').option("header","true").csv("data-resources/asset_ranking_csv")

In [28]:
## Ruta donde se encuentra el nuevo csv:

mdt_path = "data-resources/asset_ranking_csv/part-00000-b95a05be-c938-4f1b-b4fd-5c3490966a8e-c000.csv"
median_down_pd = pd.read_csv(mdt_path)
median_down_pd

,closing_id,asset_id,top_rank
0,1,6,1.0
1,1,16,2.0
2,1,14,3.0
3,1,25,4.0
4,1,23,5.0
5,1,2,6.0
6,1,24,7.0
7,1,5,8.0
8,1,51,9.0
9,1,43,10.0
